# Exploratory Data Analysis - Freelancer Earnings Dataset

В этом ноутбуке проведем первичный анализ данных о доходах фрилансеров.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set_style('whitegrid')

## Загрузка данных

In [ ]:
# Загрузка данных
df = pd.read_csv('../data/freelancer_earnings_bd.csv')
print(f"Dataset shape: {df.shape}")

## Базовая информация о данных

In [ ]:
# Просмотр первых нескольких строк
df.head()

In [ ]:
# Информация о типах данных и пропущенных значениях
df.info()

In [ ]:
# Базовые статистики числовых колонок
df.describe()

## Анализ каждой колонки

In [ ]:
# Анализ категориальных переменных
print("Job_Category:")
print(df['Job_Category'].value_counts())
print("\nPlatform:")
print(df['Platform'].value_counts())
print("\nExperience_Level:")
print(df['Experience_Level'].value_counts())
print("\nClient_Region:")
print(df['Client_Region'].value_counts())
print("\nPayment_Method:")
print(df['Payment_Method'].value_counts())
print("\nProject_Type:")
print(df['Project_Type'].value_counts())

In [ ]:
# Проверка пропущенных значений
print("Missing values per column:")
print(df.isnull().sum())

## Визуализация распределений

In [ ]:
# Распределение Earnings_USD
plt.figure(figsize=(10, 6))
sns.histplot(df['Earnings_USD'], kde=True)
plt.title('Distribution of Freelancer Earnings (USD)')
plt.xlabel('Earnings (USD)')
plt.ylabel('Count')
plt.show()

In [ ]:
# Распределение Hourly_Rate
plt.figure(figsize=(10, 6))
sns.histplot(df['Hourly_Rate'], kde=True)
plt.title('Distribution of Hourly Rates')
plt.xlabel('Hourly Rate (USD)')
plt.ylabel('Count')
plt.show()

In [ ]:
# Распределение Job_Completed
plt.figure(figsize=(10, 6))
sns.histplot(df['Job_Completed'], kde=True)
plt.title('Distribution of Jobs Completed')
plt.xlabel('Number of Jobs')
plt.ylabel('Count')
plt.show()

## Анализ взаимосвязей между переменными

In [ ]:
# Корреляционная матрица числовых переменных
numeric_df = df.select_dtypes(include=[np.number])
corr = numeric_df.corr()

plt.figure(figsize=(12, 10))
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Matrix of Numeric Variables')
plt.show()

In [ ]:
# Средний доход по методам оплаты
plt.figure(figsize=(10, 6))
sns.barplot(x='Payment_Method', y='Earnings_USD', data=df)
plt.title('Average Earnings by Payment Method')
plt.xlabel('Payment Method')
plt.ylabel('Average Earnings (USD)')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Средний доход по регионам клиентов
plt.figure(figsize=(12, 6))
sns.barplot(x='Client_Region', y='Earnings_USD', data=df)
plt.title('Average Earnings by Client Region')
plt.xlabel('Client Region')
plt.ylabel('Average Earnings (USD)')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Средний доход по уровню опыта
plt.figure(figsize=(10, 6))
sns.barplot(x='Experience_Level', y='Earnings_USD', data=df)
plt.title('Average Earnings by Experience Level')
plt.xlabel('Experience Level')
plt.ylabel('Average Earnings (USD)')
plt.show()

## Анализ данных для примеров из задания

In [ ]:
# 1. Насколько выше доход у фрилансеров, принимающих оплату в криптовалюте, по сравнению с другими способами оплаты?
crypto_earnings = df[df['Payment_Method'] == 'Cryptocurrency']['Earnings_USD'].mean()
other_earnings = df[df['Payment_Method'] != 'Cryptocurrency']['Earnings_USD'].mean()
difference = crypto_earnings - other_earnings
percentage = (difference / other_earnings) * 100

print(f"Средний доход фрилансеров, принимающих оплату в криптовалюте: ${crypto_earnings:.2f}")
print(f"Средний доход фрилансеров, использующих другие способы оплаты: ${other_earnings:.2f}")
print(f"Разница: ${difference:.2f} ({percentage:.2f}%)")

In [ ]:
# 2. Как распределяется доход фрилансеров в зависимости от региона проживания?
region_earnings = df.groupby('Client_Region')['Earnings_USD'].agg(['mean', 'median', 'std', 'count']).reset_index()
region_earnings = region_earnings.sort_values(by='mean', ascending=False)
region_earnings

In [ ]:
# 3. Какой процент фрилансеров, считающих себя экспертами, выполнил менее 100 проектов?
experts = df[df['Experience_Level'] == 'Expert']
experts_less_than_100 = experts[experts['Job_Completed'] < 100]
percentage = (len(experts_less_than_100) / len(experts)) * 100

print(f"Всего экспертов: {len(experts)}")
print(f"Экспертов с менее чем 100 проектами: {len(experts_less_than_100)}")
print(f"Процент: {percentage:.2f}%")

## Схема данных (для использования в промпте LLM)

In [ ]:
# Получение схемы данных для использования в промпте
schema = []
for col in df.columns:
    dtype = str(df[col].dtype)
    unique_values = None
    
    if df[col].dtype == 'object':
        unique_values = df[col].unique().tolist()
        if len(unique_values) > 10:  # Если слишком много уникальных значений
            unique_values = f"{len(unique_values)} уникальных значений"
    
    schema.append({
        'name': col,
        'type': dtype,
        'unique_values': unique_values
    })

for item in schema:
    print(f"{item['name']} ({item['type']})")
    if item['unique_values'] is not None and isinstance(item['unique_values'], list):
        print(f"  Unique values: {', '.join(map(str, item['unique_values']))}")
    elif item['unique_values'] is not None:
        print(f"  {item['unique_values']}")
    print()